In [6]:
import pandas as pd

# Loading the data from CSV files
data = pd.read_csv('blog_viewerpreference_202405090912.csv')

# Creating the input_data DataFrame with correctly formatted 'finger_print_id'
input_data = pd.DataFrame({
    'finger_print_id': ['77d2b1f001cedff9a9a182fdda0c3cac', '77d2b1f001cedff9a9a182fdda0c3cac', '77d2b1f001cedff9a9a182fdda0c3cac'],
    'post_id': [6, 56, 8]
})

# Printing the dataframes to verify
print("Data DataFrame:")
print(data.head())
print("\nInput Data DataFrame:")
print(input_data.head())


Data DataFrame:
   id preference_type  view_count                     created_at  \
0  30           TOPIC           1  2024-05-08 10:10:02.052 +0700   
1   1            POST           2  2024-04-08 16:46:09.275 +0700   
2   2           TOPIC           2  2024-04-08 16:46:09.289 +0700   
3   3           TOPIC           2  2024-04-08 16:46:09.299 +0700   
4  31           TOPIC           1  2024-05-08 10:10:02.064 +0700   

                      updated_at  post_id  topic_id  user_id  \
0  2024-05-08 10:10:02.057 +0700      NaN      55.0      NaN   
1  2024-04-08 16:49:20.785 +0700   1721.0       NaN    185.0   
2  2024-04-08 16:49:20.800 +0700      NaN       6.0    185.0   
3  2024-04-08 16:49:20.818 +0700      NaN      56.0    185.0   
4  2024-05-08 10:10:02.070 +0700      NaN      32.0      NaN   

                    finger_print_id  
0  77d2b1f001cedff9a9a182fdda0c3cac  
1                               NaN  
2                               NaN  
3                               NaN  


In [7]:
def calculate_similarity(input_data, data):
    similar_users = {}
    user_posts = data.groupby('finger_print_id')['post_id'].apply(set).to_dict()

    for input_user_id in input_data['finger_print_id'].unique():
        input_user_posts = set(input_data[input_data['finger_print_id'] == input_user_id]['post_id'])
        similar_users[input_user_id] = []

        for user_id, posts in user_posts.items():
            common_posts = input_user_posts.intersection(posts)
            if common_posts:
                similarity_score = len(common_posts) / len(input_user_posts.union(posts))
                similar_users[input_user_id].append((user_id, similarity_score))

    return similar_users

def find_top_similar_user(similar_users, top_n=1):
    top_similar_users = {}

    for input_user_id, user_similarities in similar_users.items():
        user_similarities.sort(key=lambda x: x[1], reverse=True)
        top_similar_users[input_user_id] = user_similarities[:top_n]

    return top_similar_users

def recommend_posts(input_data, similar_users, data):
    recommendations = {}

    for input_user_id, similar_user_list in similar_users.items():
        recommendations[input_user_id] = []

        for user_id, similarity_score in similar_user_list:
            user_posts = set(data[data['finger_print_id'] == user_id]['post_id'])
            input_user_posts = set(input_data[input_data['finger_print_id'] == input_user_id]['post_id'])
            recommended_posts = user_posts - input_user_posts
            recommendations[input_user_id].extend(recommended_posts)

    return recommendations


In [8]:
similarities = calculate_similarity(input_data, data)
top_similar_users = find_top_similar_user(similarities, top_n=1)
recommendations = recommend_posts(input_data, top_similar_users, data)

print("------------------Top similar users-----------------------\n")
for input_user_id, similar_user_list in top_similar_users.items():
    print(f"user_id {input_user_id} has top similar to:")
    for user_id, similarity_score in similar_user_list:
        print(f"  user_id {user_id} with similarity score: {similarity_score},\n")
# Additional print statements
print("------------------Matching similar personalize-----------------------")
for input_user_id, similar_user_list in top_similar_users.items():
    for user_id, similarity_score in similar_user_list:
        print(f"Input user_id {input_user_id} has post_ids: {set(input_data[input_data['user_id'] == input_user_id]['post_id'])}")
        print(f"Matching user_id {user_id} has post_ids: {set(data[data['user_id'] == user_id]['post_id'])}")
        print("\n")
print("Algorithm User Based ------>  Recommendations personalize:\n")
recommended_posts_listed = []

for input_user_id, recommended_posts in recommendations.items():
    print(f"user_id {input_user_id} should recommend the following posts:")
    print(recommended_posts)
    recommended_posts_listed.extend(recommended_posts)

print("\nRecommended posts All listed:", recommended_posts_listed)
from collections import Counter   
# Dictionary to store matching user's post_ids
matching_post_ids = {}

# Collect matching user's post_ids
for input_user_id, similar_user_list in top_similar_users.items():
    for user_id, similarity_score in similar_user_list:
        input_user_post_ids = set(input_data[input_data['finger_print_id'] == input_user_id]['post_id'])
        matching_user_post_ids = set(data[data['user_id'] == user_id]['post_id'])
        matching_post_ids[input_user_id] = input_user_post_ids.intersection(matching_user_post_ids)

# Find the post_id with the most users among matching users
most_common_post_id = Counter([post_id for post_ids in matching_post_ids.values() for post_id in post_ids]).most_common(1)[0][0]



------------------Top similar users-----------------------

user_id 77d2b1f001cedff9a9a182fdda0c3cac has top similar to:
------------------Matching similar personalize-----------------------
Algorithm User Based ------>  Recommendations personalize:

user_id 77d2b1f001cedff9a9a182fdda0c3cac should recommend the following posts:
[]

Recommended posts All listed: []


IndexError: list index out of range